In [23]:
from keras.models import load_model
import numpy as np
import pandas as pd
import pickle

In [24]:
with open('encoders.pkl', 'rb') as f:
    loaded_encoders = pickle.load(f)

user_encoder = loaded_encoders['user_encoder']    
anime_encoder = loaded_encoders['anime_encoder']

In [2]:
model = load_model('myanimemodel.h5')

In [3]:
def extract_weights(name, model):
    # Get the layer by name from the model
    weight_layer = model.get_layer(name)
    
    # Get the weights from the layer
    weights = weight_layer.get_weights()[0]
    
    # Normalize the weights
    weights = weights / np.linalg.norm(weights, axis=1).reshape((-1, 1))
    
    return weights

# Extract weights for anime embeddings
anime_weights = extract_weights('anime_embedding', model)
# Extract weights for user embeddings
user_weights = extract_weights('user_embedding', model)

In [5]:
len(anime_weights[0])

128

In [8]:
df_anime = pd.read_csv('data/anime-dataset-2023.csv')

In [87]:
popularity_threshold = 50 # This treshold is to filter out anime rated by less than 50 people
df_anime= df_anime.query('Members >= @popularity_threshold')
print(df_anime.shape)

(22879, 24)


In [32]:
anime_row = df_anime[df_anime['Name'] == 'Trigun'].iloc[0]
index = anime_row['anime_id']
encoded_index = anime_encoder.transform([index])[0]
weights = anime_weights
dists = np.dot(weights, weights[encoded_index])

array([ 0.66890293,  0.61240506,  0.99999994, ..., -0.0074106 ,
       -0.04696456, -0.06391112], dtype=float32)

In [85]:
def find_similar_animes(name, n=10):
    
    try:
        
        anime_row = df_anime[df_anime['Name'] == name].iloc[0]
        
        index = anime_row['anime_id']
        
        encoded_index = anime_encoder.transform([index])[0]
        
        weights = anime_weights
        
        dists = np.dot(weights, weights[encoded_index])
        
        sorted_dists = np.argsort(dists)
        
        n = n + 1
        
        closest = sorted_dists[-n:]
        
        SimilarityArr = []
        
        for close in closest:
            decoded_id = anime_encoder.inverse_transform([close])[0]
            anime_frame = df_anime[df_anime['anime_id'] == decoded_id]
            
            anime_name = anime_frame['Name'].values[0]
            english_name = anime_frame['English name'].values[0]
            name = english_name if english_name != "UNKNOWN" else anime_name
            genre = anime_frame['Genres'].values[0]
            Synopsis = anime_frame['Synopsis'].values[0]
            similarity = dists[close]
            similarity = "{:.2f}%".format(similarity * 100)
            SimilarityArr.append({"Name": name, "Similarity": similarity, "Genres": genre, "Synopsis":Synopsis})
        Frame = pd.DataFrame(SimilarityArr).sort_values(by="Similarity", ascending=False)
        return Frame[Frame.Name != name]
        
        return closest
            
    except:
        print(f'{name} not found')
        
#pd.set_option('display.max_colwidth', None)

In [81]:
find_similar_animes('Death Note')

,Name,Similarity,Genres,Synopsis
9,Code Geass: Lelouch of the Rebellion R2,89.85%,"Action, Award Winning, Drama, Sci-Fi","One year has passed since the Black Rebellion, a failed uprising against the Holy Britannian Empire led by the masked vigilante Zero, who is now missing. At a loss without their revolutionary leader, Area 11's resistance group—the Black Knights—find themselves too powerless to combat the brutality inflicted upon the Elevens by Britannia, which has increased significantly in order to crush any hope of a future revolt. \n\nLelouch Lamperouge, having lost all memory of his double life, is living peacefully alongside his friends as a high school student at Ashford Academy. His former partner C.C., unable to accept this turn of events, takes it upon herself to remind him of his past purpose, hoping that the mastermind Zero will rise once again to finish what he started, in this thrilling conclusion to the series."
8,Code Geass: Lelouch of the Rebellion,89.20%,"Action, Award Winning, Drama, Sci-Fi","In the year 2010, the Holy Empire of Britannia is establishing itself as a dominant military nation, starting with the conquest of Japan. Renamed to Area 11 after its swift defeat, Japan has seen significant resistance against these tyrants in an attempt to regain independence.\n\nLelouch Lamperouge, a Britannian student, unfortunately finds himself caught in a crossfire between the Britannian and the Area 11 rebel armed forces. He is able to escape, however, thanks to the timely appearance of a mysterious girl named C.C., who bestows upon him Geass, the ""Power of Kings."" Realizing the vast potential of his newfound ""power of absolute obedience,"" Lelouch embarks upon a perilous journey as the masked vigilante known as Zero, leading a merciless onslaught against Britannia in order to get revenge once and for all."
7,Fullmetal Alchemist: Brotherhood,84.36%,"Action, Adventure, Drama, Fantasy","After a horrific alchemy experiment goes wrong in the Elric household, brothers Edward and Alphonse are left in a catastrophic new reality. Ignoring the alchemical principle banning human transmutation, the boys attempted to bring their recently deceased mother back to life. Instead, they suffered brutal personal loss: Alphonse's body disintegrated while Edward lost a leg and then sacrificed an arm to keep Alphonse's soul in the physical realm by binding it to a hulking suit of armor.\n\nThe brothers are rescued by their neighbor Pinako Rockbell and her granddaughter Winry. Known as a bio-mechanical engineering prodigy, Winry creates prosthetic limbs for Edward by utilizing ""automail,"" a tough, versatile metal used in robots and combat armor. After years of training, the Elric brothers set off on a quest to restore their bodies by locating the Philosopher's Stone—a powerful gem that allows an alchemist to defy the traditional laws of Equivalent Exchange.\n\nAs Edward becomes an infamous alchemist and gains the nickname ""Fullmetal,"" the boys' journey embroils them in a growing conspiracy that threatens the fate of the world."
6,Attack on Titan,77.65%,"Action, Award Winning, Drama, Suspense","Centuries ago, mankind was slaughtered to near extinction by monstrous humanoid creatures called Titans, forcing humans to hide in fear behind enormous concentric walls. What makes these giants truly terrifying is that their taste for human flesh is not born out of hunger but what appears to be out of pleasure. To ensure their survival, the remnants of humanity began living within defensive barriers, resulting in one hundred years without a single titan encounter. However, that fragile calm is soon shattered when a colossal Titan manages to breach the supposedly impregnable outer wall, reigniting the fight for survival against the man-eating abominations.\n\nAfter witnessing a horrific personal loss at the hands of the invading creatures, Eren Yeager dedicates his life to their eradication by enlisting into the Survey Corps, an elite mil